In [1]:
import json
import os
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

In [18]:
files = os.listdir('tvk')
first = True

for file in tqdm_notebook(files):
    with open(os.path.join('tvk',file),'r', encoding="utf8") as cur_file:
        parsed_file = json.load(cur_file)
        if first:
            tvos_all = parsed_file
            first = False
        else:
            tvos_all['features'] = tvos_all['features'] + parsed_file['features']
            
len(tvos_all['features'])

225

In [3]:
dtype_dic = {'district_number': str}
df_results = pd.read_csv('00_data/elections_round2_sim_v6.csv',dtype=dtype_dic)

In [16]:
df_results['participation_absolute_round1'] = df_results['2.К-сть виборців, внесених до списку']*df_results['participation_round1']
df_results['participation_absolute_round2'] = df_results['2.К-сть виборців, внесених до списку']*df_results['participation_round2']
df_results_tvo = df_results.groupby('district_gen_numeric')[['2.К-сть виборців, внесених до списку',
                                                             'participation_absolute_round1',
                                                             'participation_absolute_round2',
                                                             'ЗеленськийВолодимир_round1',
                                                             'ПорошенкоПетро_round1',
                                                             'ЗеленськийВолодимир',
                                                             'ПорошенкоПетро']].sum()
df_results_tvo['participation_diff'] = (df_results_tvo['participation_absolute_round2']-df_results_tvo['participation_absolute_round1'])/df_results_tvo['2.К-сть виборців, внесених до списку']
df_results_tvo['winner'] = (df_results_tvo['ЗеленськийВолодимир'] - df_results_tvo['ПорошенкоПетро']).map(lambda x: 'zel' if x>0 else 'por')
df_results_tvo['zel_growth'] = (df_results_tvo['ЗеленськийВолодимир'] - df_results_tvo['ЗеленськийВолодимир_round1'])/df_results_tvo['2.К-сть виборців, внесених до списку']
df_results_tvo['por_growth'] = (df_results_tvo['ПорошенкоПетро'] - df_results_tvo['ПорошенкоПетро_round1'])/df_results_tvo['2.К-сть виборців, внесених до списку']

print(df_results_tvo.shape)
df_results_tvo.head()

(200, 11)


,"2.К-сть виборців, внесених до списку",participation_absolute_round1,participation_absolute_round2,ЗеленськийВолодимир_round1,ПорошенкоПетро_round1,ЗеленськийВолодимир,ПорошенкоПетро,participation_diff,winner,zel_growth,por_growth
district_gen_numeric,,,,,,,,,,,
11,169689,115238.0,113211.243534,26398,33083,63840,49374,-0.011944,zel,0.220651,0.096005
12,177815,115722.0,114846.839503,28535,31819,62401,52440,-0.004922,zel,0.190456,0.115969
13,145583,95116.0,93348.125547,22164,20460,51149,42199,-0.012143,zel,0.199096,0.149324
14,159494,102258.0,101666.824533,24356,23232,54497,47171,-0.003707,zel,0.188979,0.150093
15,148528,93001.0,92313.772869,21896,18318,50707,41606,-0.004627,zel,0.193977,0.156792


In [19]:
for feature in tqdm_notebook(tvos_all['features']):
    if int(feature['properties']['id']) in df_results_tvo.index:
        feature['properties']['participation_diff'] = df_results_tvo.loc[int(feature['properties']['id'])]['participation_diff']
        feature['properties']['zel_growth'] = df_results_tvo.loc[int(feature['properties']['id'])]['zel_growth']
        feature['properties']['por_growth'] = df_results_tvo.loc[int(feature['properties']['id'])]['por_growth']
        feature['properties']['winner'] = df_results_tvo.loc[int(feature['properties']['id'])]['winner']
    else:
        feature['properties']['participation_diff'] = 9999
        feature['properties']['zel_growth'] = 9999
        feature['properties']['por_growth'] = 9999
        feature['properties']['winner'] = 'no_vote'

len(tvos_all['features'])

225

In [20]:
with open('01_map/tvos_all_v4.json','w', encoding="utf8") as output:
    json.dump(tvos_all,output,ensure_ascii=False)

In [84]:
with open('newElectionData.json','r',encoding="utf8") as file:
    districts_all = json.load(file)

In [118]:
district = districts_all['objects']['-']['geometries'][1300]

for district in tqdm_notebook(districts_all['objects']['-']['geometries']):
    if district['properties']['d'] in list(df_results['district_number']):
        district['properties']['participation_diff'] = df_results.loc[df_results['district_number']==district['properties']['d']].reset_index()['participation_delta'][0]
    else:
        district['properties']['participation_diff'] = 9999

In [126]:
with open('newElectionData_v2.json','w',encoding="utf8") as file:
    json.dump(districts_all,file,ensure_ascii=False)